This is my first Kaggle Kernel so I am always open to feedback please upvote if you enjoy, hopefully it will be the first kernel of many.


# Iterating RANZCR Baselines in a Flash


## What is Flash?

![](https://miro.medium.com/max/1400/1*zF-Uy9kX-Fe38_NjE8hhSw.png)
PyTorch Lightning Flash is a collection of tasks for fast prototyping, baselining and fine-tuning scalable Deep Learning models, built on PyTorch Lightning. This Kernel shows to go from simple baseline defaults < 15 lines of code to fine-tuning more complex state of the art models with complex augmenations using flash.


Check out this post describing [Flash](https://medium.com/pytorch/introducing-lightning-flash-the-fastest-way-to-get-started-with-deep-learning-202f196b3b98) and the repo on [GitHub](https://github.com/PyTorchLightning/lightning-flash) for more info.



## Getting Started With a Baseline

Let's get started by creating a Flash baseline for the Ranzcr challenge with about 15 lines of code using the default Resnet 18 configuration.


### Install Flash

In [ ]:
!pip install git+https://github.com/aribornstein/lightning-flash.git@flash_multilabel_clf_aug

In [ ]:
import os
from flash.data import labels_from_csv
from flash.vision import ImageClassificationData
from flash.vision import ImageClassifier
from flash import Trainer
from torch.nn import functional as F


root = '../input/ranzcr-clip-catheter-line-classification'
columns = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
           'NGT - Abnormal','NGT - Borderline','NGT - Incompletely Imaged','NGT - Normal',
           'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

# get train dataloader
data = ImageClassificationData.from_filepaths(
    train_filepaths=os.path.join(root, 'train'),
    train_labels= labels_from_csv(os.path.join(root, 'train.csv'),'StudyInstanceUID', representation='onehot', feature_cols=columns),
    valid_split=0.10)

model = ImageClassifier(multilabel=True, num_classes=len(columns), loss_fn=F.binary_cross_entropy_with_logits)

## Fine Tune With 1 GPU
trainer = Trainer(gpus=1, max_epochs=1)
trainer.finetune(model, data, strategy='no_freeze')

### That's all we needed to do to get a baseline model. Not bad for less than 15 lines of code. Now granted this model isn't getting on a leader board anytime soon but we can do much better let's play around with some complex state of the art backbones using the [Timm package](https://github.com/rwightman/pytorch-image-models).


## State of the Art with Timm Resnet200d and ViT

In [ ]:
!pip install timm

In [ ]:
import timm
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl


### Custom Resnet200d Backbone

In [ ]:
model = timm.create_model('resnet200d', pretrained=True)
model.global_pool = torch.nn.Identity()
model.fc = torch.nn.Identity()
pooling = torch.nn.AdaptiveAvgPool2d(1)
backbone = (model, model.num_features)

In [ ]:
model = ImageClassifier(backbone=backbone, # use resnet200d backbone
                        optimizer = torch.optim.Adam, # Use Adam instead of SGD
                        loss_fn=F.binary_cross_entropy_with_logits,
                        multilabel=True,
                        num_classes=len(columns))


In [ ]:
## Fine Tune With 1 GPU
trainer = Trainer(
    gpus=1, 
    max_epochs=1,
    auto_lr_find=True,# auto find learning rate     
    precision=16, # use 16 bit precision  
    auto_scale_batch_size='binsearch', # maximize batch size to fit device memory
    callbacks=[EarlyStopping(monitor='val_binary_cross_entropy_with_logits')] # early stopping
)

trainer.finetune(model, data, strategy='freeze')

### Custom ViT Backbone

In [ ]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)
backbone = (model, model.num_features)

In [ ]:
model = ImageClassifier(backbone=backbone, # use resnet200d backbone
                        optimizer = torch.optim.Adam, # Use Adam instead of SGD
                        loss_fn=F.binary_cross_entropy_with_logits,
                        multilabel=True,
                        num_classes=len(columns))

In [ ]:
## Fine Tune With 1 GPU
trainer = Trainer(
    gpus=1, 
    max_epochs=1,
    auto_lr_find=True,# auto find learning rate     
    precision=16, # use 16 bit precision  
    auto_scale_batch_size='binsearch', # maximize batch size to fit device memory
    callbacks=[EarlyStopping(monitor='val_binary_cross_entropy_with_logits')] # early stopping
)

trainer.finetune(model, data, strategy='freeze')

## Adding Data Augmenation with [Albumentations](https://github.com/albumentations-team/albumentations)
![](https://camo.githubusercontent.com/3bb6e4bb500d96ad7bb4e4047af22a63ddf3242a894adf55ebffd3e184e4d113/68747470733a2f2f686162726173746f726167652e6f72672f776562742f62642f6e652f72762f62646e6572763563746b75646d73617a6e687734637273646669772e6a706567)


Now we are making progress let's add some data augmentations using the Albumentations library insipred by this [repo by VietHoang1710](https://github.com/VietHoang1710/RANZCR_2021)

In [ ]:
image_size = 700
train_transform = albumentations.Compose(
            [
                albumentations.RandomResizedCrop(height=image_size, width=image_size, scale=(0.9, 1), p=1),
                albumentations.HorizontalFlip(p=0.5),
                albumentations.ShiftScaleRotate(p=0.5),
                albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.7
                ),
                albumentations.CLAHE(clip_limit=(1, 4), p=0.5),
                albumentations.OneOf(
                    [
                        albumentations.OpticalDistortion(distort_limit=1.0),
                        albumentations.GridDistortion(num_steps=5, distort_limit=1.0),
                        albumentations.ElasticTransform(alpha=3),
                    ],
                    p=0.2,
                ),
                albumentations.OneOf(
                    [
                        albumentations.GaussNoise(var_limit=[10, 50]),
                        albumentations.GaussianBlur(),
                        albumentations.MotionBlur(),
                        albumentations.MedianBlur(),
                    ],
                    p=0.2,
                ),
                albumentations.OneOf(
                    [
                        albumentations.JpegCompression(),
                        albumentations.Downscale(scale_min=0.1, scale_max=0.15),
                    ],
                    p=0.2,
                ),
                albumentations.Resize(height=image_size, width=image_size),
                albumentations.IAAPiecewiseAffine(p=0.2),
                albumentations.IAASharpen(p=0.2),
                albumentations.Cutout(
                    max_h_size=int(image_size * 0.1),
                    max_w_size=int(image_size * 0.1),
                    num_holes=5,
                    p=0.5,
                ),
                albumentations.Normalize(),
            ]
)

transform_val = albumentations.Compose(
    [
        albumentations.Resize(height=image_size, width=image_size),
        albumentations.Normalize(),
    ]
)

In [ ]:
# get train dataloader with transforms 
data = ImageClassificationData.from_filepaths(
    train_transform = train_transform,
    valid_transform = transform_val,
    batch_size=32,
    train_filepaths=os.path.join(root, 'train'),
    train_labels=train_labels,
    valid_split=0.10,
    num_workers=4
)

In [ ]:
## Fine Tune With 1 GPU
trainer = Trainer(
    gpus=1, 
    max_epochs=1,
    auto_lr_find=True,# auto find learning rate     
    precision=16, # use 16 bit precision  
    auto_scale_batch_size='binsearch', # maximize batch size to fit device memory
    callbacks=[EarlyStopping(monitor='val_binary_cross_entropy_with_logits')] # early stopping
)

trainer.finetune(model, data, strategy='freeze')

## Predict

Now lets run our model on the test set. Future versions of Flash will make this process even cleaner.

In [ ]:
from tqdm.notebook import tqdm 
import pandas as pd 

# list of files to predict
test_path = os.path.join(root, 'test')
test_files = os.listdir(test_path)
test_files = [os.path.join(test_path, x) for x in test_files]

# make the predictions in batches

preds = []
batch_size = 64
print("Predicting")
for i in tqdm(range(0, len(test_files), batch_size)):
    end_i = min(i + batch_size, len(test_files))
    batch_file_paths = test_files[i: end_i]
    batch_preds = clf.predict(batch_file_paths)
    preds.extend(batch_preds)
    
# read the names of the test files
test_file_dir = os.path.join(root, 'test')
test_file_names = os.listdir(test_file_dir)
test_file_names = [os.path.splitext(x)[0] for x in test_file_names]
pred_csv = pd.DataFrame(test_file_names, columns=['StudyInstanceUID'])
pred_df = pd.DataFrame(preds, columns=columns)
pred_csv = pd.concat([pred_csv, pred_df], axis=1)
pred_csv.to_csv(os.path.join(output, 'submission.csv'), index=False)

## Conclusions

Hopefully you now see how easy flash makes it to baseline and iterate image classificaiton tasks powered by PyTorch Lightning under the hood.

I want to give a huge thanks to the authors of Flash, Timm and Albumentations you should star each of these repos to show your support and thank you a huge shout out to these kernels that helped insipire me please up vote them.

- Sin's https://www.kaggle.com/underwearfitting/resnet200d-public-benchmark-2xtta-lb0-965/data?scriptVersionId=51087772
- Ammarali32's https://www.kaggle.com/ammarali32/resnet200d-inference-single-model-lb-96-5/data
- AshishGupta https://www.kaggle.com/roydatascience/resnet200d-public-benchmark-inference-model

Data Analysis

- [https://www.kaggle.com/isaienkov/ranzcr-clip-data-understanding](https://www.kaggle.com/isaienkov/ranzcr-clip-data-understanding)
- [https://www.kaggle.com/amitalexander/ranzcr-clip-exploratory-data-analysis](https://www.kaggle.com/amitalexander/ranzcr-clip-exploratory-data-analysis)
- [https://www.kaggle.com/parthdhameliya77/ranzcr-clip-eda-class-imbalance-patient-overlap](https://www.kaggle.com/parthdhameliya77/ranzcr-clip-eda-class-imbalance-patient-overlap)

Data Preprocessing 

- Get rid of patient overlap between test and validation data [https://www.kaggle.com/parthdhameliya77/ranzcr-clip-eda-class-imbalance-patient-overlap](https://www.kaggle.com/parthdhameliya77/ranzcr-clip-eda-class-imbalance-patient-overlap)
- Making images more clear [https://www.kaggle.com/aryaman1999/clearing-the-fog-outta-those-catheters](https://www.kaggle.com/aryaman1999/clearing-the-fog-outta-those-catheters)
- Segmenting out the catheter lines - [https://www.kaggle.com/ryches/segmentation-model](https://www.kaggle.com/ryches/segmentation-model)
- Make sure folds are stratified [https://www.kaggle.com/virilo/ranzcr-clip-stratified-kfold-to-team-up-v3/](https://www.kaggle.com/virilo/ranzcr-clip-stratified-kfold-to-team-up-v3/)



## About the Author

Aaron (Ari) Bornstein is an AI researcher with a passion for history, engaging with new technologies and computational medicine. As Head of Developer Advocacy at Grid.ai, he collaborates with the Machine Learning Community to solve real-world problems with game-changing technologies that are then documented, open-sourced, and shared with the rest of the world.
